In [1]:
import chardet
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
today = date.today()
import mysql.connector

In [2]:
cnx = mysql.connector.connect(user='reporte', password='20rem07',
                              host='192.168.0.3',
                              database='remesa_remesa')

In [3]:
sql = '''SELECT 
  clie.CL_NOMFANTASIA,
  funcionario.FU_NOMBRE as EJECUTIVO,
  DATE_FORMAT(dr.DR_FECHA, '%d-%m-%Y') as DR_FECHA,
  deu.DD_RUT,
  DATE_FORMAT(pd.PG_FECHA, '%d-%m-%Y') as PG_FECHA,
  REPLACE(pd.PG_VALOR,".0000","") as PG_VALOR,
  REPLACE(d.DC_MONTO,".0000","") as DC_MONTO,
  REPLACE(d.DC_SALDO,".0000","") as DC_SALDO
  FROM pagodoc pd
  INNER JOIN documento AS d ON d.DC_ID = pd.DC_ID
  INNER JOIN dilres_ultimas_dil dr ON dr.DC_ID = d.DC_ID 
  INNER JOIN cliente as clie ON clie.CL_RUT = d.CL_RUT
  INNER JOIN deudor as deu ON d.DD_RUT=deu.DD_RUT and d.CL_RUT=deu.CL_RUT
  INNER JOIN funcionario ON funcionario.FU_CODIGO =d.ID_USUARIO
  WHERE d.ELIMINADO = 0
  AND d.CL_RUT IN ('1-9','2-7','3-5','8-6','70-1','76.914.254-1')
  ORDER BY d.DC_ID ASC;
'''

In [4]:
df = pd.read_sql(sql, cnx, coerce_float=True)

In [5]:
df.PG_VALOR = pd.to_numeric(df['PG_VALOR']).astype(int)
df.DC_MONTO = pd.to_numeric(df['DC_MONTO']).astype(int) 
df.DC_SALDO = pd.to_numeric(df['DC_SALDO']).astype(int)
df.PG_FECHA = pd.to_datetime(df['PG_FECHA'], errors='coerce')
df.DR_FECHA = pd.to_datetime(df['PG_FECHA'], errors='coerce')

In [6]:
df.CL_NOMFANTASIA.value_counts()

REMESA 5          52596
REMESA 7          49246
REMESA 6          22489
REMESA 8           8112
REMESA 8 IVAN      2141
REMESA 8 CAROL     1808
Name: CL_NOMFANTASIA, dtype: int64

In [7]:
df[df.EJECUTIVO=='EMPEX HMCIA'].PG_VALOR.sum(), df[df.EJECUTIVO=='UNICOB'].PG_VALOR.sum()

(20141001, 15939618)